In [4]:
from langchain_core.prompts import PromptTemplate


In [5]:
from langchain_community.llms import Tongyi
model=Tongyi(model='qwen-plus')

In [6]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# 自定义的文本拆分器
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=30,separators=['第'])
loader=TextLoader("data/extracted_text.txt",encoding="utf-8")

docs=loader.load_and_split(text_splitter=text_splitter)

In [7]:
from langchain_chroma import Chroma
from langchain_community.embeddings import DashScopeEmbeddings
vectorstore = Chroma.from_documents(documents=docs, embedding=DashScopeEmbeddings(model="text-embedding-v3"))


检索

In [8]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})


In [9]:
template="你是一个供电局的客服人员，下面是用户的问题和相关的知识库内容，请根据知识库内容回答用户的问题。\n\n问题：{question}\n\n知识库：{docs}\n\n回答："
prompt = PromptTemplate(template=template, input_variables=["question","docs"])


In [16]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain =  prompt| model| StrOutputParser()

In [ ]:
question='供电企业用户受电端的供电电压偏差是多少。并列出检索自第几条？'
formatted_docs = (retriever | format_docs).invoke(question)
rag_chain.invoke({f"question":question 	, "docs": formatted_docs})

'根据知识库内容，供电企业用户受电端的供电电压偏差规定如下：\n\n1. **35千伏以上电压供电的**：电压正、负偏差的绝对值之和不超过额定值的10%。\n2. **10（6、20）千伏以下三相供电的**：为额定值的±7%。\n3. **220伏单相供电的**：为额定值的+7%、－10%。\n\n在电力系统非正常状况下，用户受电端的电压最大允许偏差不应超过额定值的±10%。\n\n检索自**第五十七条**。'

'根据知识库内容，以下是关于供电电压允许偏差的回答：\n\n在电力系统正常状况下，供电企业供到用户受电端的供电电压允许偏差为（**第五十七条**）：\n\n1. **35千伏以上电压供电的**：电压正、负偏差的绝对值之和不超过额定值的 **10%**。\n2. **10（6、20）千伏以下三相供电的**：为额定值的 **±7%**。\n3. **220伏单相供电的**：为额定值的 **+7%、－10%**。\n\n此外，在电力系统非正常状况下，用户受电端的电压最大允许偏差不应超过额定值的 **±10%**。 \n\n如需进一步了解，请参考相关条款或联系供电局获取详细信息。'

'根据您提供的信息和知识库内容，针对您的问题“2个宅基地、2个产证但只有一个门口，可以报装2个户号吗”，以下是我们的回复：\n\n### 回答：\n**可以报装2个户号，但需满足以下条件：**\n\n1. **独立的不动产权属**  \n   根据知识库第三十二条（二），分立后的用户地址均应当具有独立的不动产权属。您提到的情况中，虽然只有1个门口，但有2个宅基地和2个产证，说明已经具备了独立的不动产权属，这是符合分户条件的。\n\n2. **受电装置具备分装条件**  \n   根据知识库第三十二条（一），在用电地址、供电点、用电容量不变的情况下，如果受电装置具备分装条件，可以办理分户。因此，需要确保现场的电力设施能够支持分开安装两套独立的计量装置。\n\n3. **与供电企业重新建立供用电关系**  \n   分户后的新用户需要与供电企业重新签订供用电合同，并分别计费。\n\n4. **原用户的债务结清**  \n   如果是现有用户申请分户，需确保原用户与供电企业之间的债务已结清（知识库第三十二条（三））。\n\n5. **工程费用由分户者承担**  \n   分户引起的用户产权范围内的工程费用需由分户者自行承担（知识库第三十二条（六））。\n\n6. **特殊情况下的处理**  \n   如果因特殊原因无法实现一户一表（如只有一个门口），根据知识库第七十八条，供电企业可以根据实际情况安装共用的计费电能表，各用户可自行装设分户电能表并分算电费。不过，这种情况需要提前与供电企业沟通确认。\n\n### 总结：\n只要您的两个宅基地分别持有独立的不动产权证书，并且电力设施具备分装条件，就可以向供电企业申请报装2个户号。建议您携带相关证明材料（如不动产权证书、身份证明等）前往当地供电营业厅咨询并办理具体手续。如有疑问，供电企业工作人员会为您提供进一步的技术指导和支持。'